In [5]:
import numpy as np

In [6]:
# Pierwszy test sprawdzenia czy wszystko ok dla prostych macierzy wypłat
def test1(M=10000):
    
    C_1 = np.array([[1,2,3],[4,5,6],[7,8,9]]) # bierzemy jakąś macierz wypłat dla następnika
    C_1 = C_1.T

    R_1 = np.array([[9,8,7],[6,5,4],[3,2,1]]) # bierzemy jakąś macierz wypłat dla lidera

    print('macierz wypłat szeryfa')
    print(R_1)
    print('macierz wypłat kupca 1')
    print(C_1)

    # Setujemy problem
    p = MixedIntegerLinearProgram(maximization=True, solver="GLPK")
    
    # Dodajemy odpowiednie zmienne
    # Konwencja: z[12_3] to z dla i=1, j=2, l=3
    # q[2_1] to q dla j=2, l=1
    # a[1] to a dla l=1
    z = p.new_variable()
    p.set_min(z, 0)
    p.set_max(z, 1)
    q = p.new_variable(binary= True)
    a = p.new_variable()

    # Kolejno od góry do dołu zapisujemy ograniczenia
    p.add_constraint(z[11_1] + z[12_1] + z[13_1] + z[21_1] + z[22_1] + z[23_1] + z[31_1] + z[32_1] + z[33_1] == 1)


    p.add_constraint(z[11_1] + z[12_1] + z[13_1] <= 1)
    p.add_constraint(z[21_1] + z[22_1] + z[23_1] <= 1)
    p.add_constraint(z[31_1] + z[32_1] + z[33_1] <= 1)


    p.add_constraint(q[1_1] <= z[11_1] + z[21_1] + z[31_1]  <= 1)
    p.add_constraint(q[2_1] <= z[12_1] + z[22_1] + z[32_1]  <= 1)
    p.add_constraint(q[3_1] <= z[13_1] + z[23_1] + z[33_1]  <= 1)


    p.add_constraint(q[1_1] + q[2_1] + q[3_1]  == 1)


    p.add_constraint(0 <= a[1] - (C_1[0,0]*(z[11_1] + z[12_1] + z[13_1]) +
                              C_1[1,0]*(z[21_1] + z[22_1] + z[23_1]) +
                              C_1[2,0]*(z[31_1] + z[32_1] + z[33_1])) <= (1-q[1_1])*M)
    p.add_constraint(0 <= a[1] - (C_1[0,1]*(z[11_1] + z[12_1] + z[13_1]) +
                              C_1[1,1]*(z[21_1] + z[22_1] + z[23_1]) +
                              C_1[2,1]*(z[31_1] + z[32_1] + z[33_1])) <= (1-q[2_1])*M)
    p.add_constraint(0 <= a[1] - (C_1[0,2]*(z[11_1] + z[12_1] + z[13_1]) +
                              C_1[1,2]*(z[21_1] + z[22_1] + z[23_1]) +
                              C_1[2,2]*(z[31_1] + z[32_1] + z[33_1])) <= (1-q[3_1])*M)
    # ponieważ L = {1} to pomijamy ostatnie ogarniczenie
    
    # Ładujemy funkcję celu
    p.set_objective( 
                    (R_1[0,0] * z[11_1] + R_1[0,1] * z[12_1] + R_1[0,2] * z[13_1]
                     + R_1[1,0] * z[21_1] + R_1[1,1] * z[22_1] + R_1[1,2] * z[23_1]
                     + R_1[2,0] * z[31_1] + R_1[2,1] * z[32_1] + R_1[2,2] * z[33_1])
                    )
    # Sprawdzamy wyniki
    p.show()
    print('Objective Value: {}'.format(p.solve()))

    print('Rozwiązanie: ')
    for i, v in sorted(p.get_values(z).items()):
        print(f'z_{i} = {v}')
    for i, v in sorted(p.get_values(q).items()):
        print(f'q_{i} = {v}')
    for i, v in sorted(p.get_values(a).items()):
        print(f'a_{i} = {v}')
    
test1()

macierz wypłat szeryfa
[[9 8 7]
 [6 5 4]
 [3 2 1]]
macierz wypłat kupca 1
[[1 4 7]
 [2 5 8]
 [3 6 9]]
Maximization:
  9.0 x_0 + 8.0 x_1 + 7.0 x_2 + 6.0 x_3 + 5.0 x_4 + 4.0 x_5 + 3.0 x_6 + 2.0 x_7 + x_8 

Constraints:
  1.0 <= x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 <= 1.0
  x_0 + x_1 + x_2 <= 1.0
  x_3 + x_4 + x_5 <= 1.0
  x_6 + x_7 + x_8 <= 1.0
  - x_0 - x_3 - x_6 + x_9 <= 0.0
  x_0 + x_3 + x_6 <= 1.0
  - x_1 - x_4 - x_7 + x_10 <= 0.0
  x_1 + x_4 + x_7 <= 1.0
  - x_2 - x_5 - x_8 + x_11 <= 0.0
  x_2 + x_5 + x_8 <= 1.0
  1.0 <= x_9 + x_10 + x_11 <= 1.0
  x_0 + x_1 + x_2 + 2.0 x_3 + 2.0 x_4 + 2.0 x_5 + 3.0 x_6 + 3.0 x_7 + 3.0 x_8 - x_12 <= 0.0
  - x_0 - x_1 - x_2 - 2.0 x_3 - 2.0 x_4 - 2.0 x_5 - 3.0 x_6 - 3.0 x_7 - 3.0 x_8 + 10000.0 x_9 + x_12 <= 10000.0
  4.0 x_0 + 4.0 x_1 + 4.0 x_2 + 5.0 x_3 + 5.0 x_4 + 5.0 x_5 + 6.0 x_6 + 6.0 x_7 + 6.0 x_8 - x_12 <= 0.0
  -4.0 x_0 - 4.0 x_1 - 4.0 x_2 - 5.0 x_3 - 5.0 x_4 - 5.0 x_5 - 6.0 x_6 - 6.0 x_7 - 6.0 x_8 + 10000.0 x_10 + x_12 <= 10000.

In [7]:
# Drugi test sprawdzenia czy wszystko ok dla prostych macierzy wypłat
def test2(M=10000):
    
    C_1 = np.array([[1,2,3],[4,5,6],[7,8,9]]) # bierzemy jakąś macierz wypłat dla następnika

    R_1 = np.array([[9,8,7],[6,5,4],[3,2,1]]) # bierzemy jakąś macierz wypłat dla lidera

    print('macierz wypłat szeryfa')
    print(R_1)
    print('macierz wypłat kupca 1')
    print(C_1)

    # Setujemy problem
    p = MixedIntegerLinearProgram(maximization=True, solver="GLPK")
    
    # Dodajemy odpowiednie zmienne
    # Konwencja: z[12_3] to z dla i=1, j=2, l=3
    # q[2_1] to q dla j=2, l=1
    # a[1] to a dla l=1
    z = p.new_variable()
    p.set_min(z, 0)
    p.set_max(z, 1)
    q = p.new_variable(binary= True)
    a = p.new_variable()

    # Kolejno od góry do dołu zapisujemy ograniczenia
    p.add_constraint(z[11_1] + z[12_1] + z[13_1] + z[21_1] + z[22_1] + z[23_1] + z[31_1] + z[32_1] + z[33_1] == 1)


    p.add_constraint(z[11_1] + z[12_1] + z[13_1] <= 1)
    p.add_constraint(z[21_1] + z[22_1] + z[23_1] <= 1)
    p.add_constraint(z[31_1] + z[32_1] + z[33_1] <= 1)


    p.add_constraint(q[1_1] <= z[11_1] + z[21_1] + z[31_1]  <= 1)
    p.add_constraint(q[2_1] <= z[12_1] + z[22_1] + z[32_1]  <= 1)
    p.add_constraint(q[3_1] <= z[13_1] + z[23_1] + z[33_1]  <= 1)


    p.add_constraint(q[1_1] + q[2_1] + q[3_1]  == 1)


    p.add_constraint(0 <= a[1] - (C_1[0,0]*(z[11_1] + z[12_1] + z[13_1]) +
                              C_1[1,0]*(z[21_1] + z[22_1] + z[23_1]) +
                              C_1[2,0]*(z[31_1] + z[32_1] + z[33_1])) <= (1-q[1_1])*M)
    p.add_constraint(0 <= a[1] - (C_1[0,1]*(z[11_1] + z[12_1] + z[13_1]) +
                              C_1[1,1]*(z[21_1] + z[22_1] + z[23_1]) +
                              C_1[2,1]*(z[31_1] + z[32_1] + z[33_1])) <= (1-q[2_1])*M)
    p.add_constraint(0 <= a[1] - (C_1[0,2]*(z[11_1] + z[12_1] + z[13_1]) +
                              C_1[1,2]*(z[21_1] + z[22_1] + z[23_1]) +
                              C_1[2,2]*(z[31_1] + z[32_1] + z[33_1])) <= (1-q[3_1])*M)
    # ponieważ L = {1} to pomijamy ostatnie ogarniczenie
    
    # Ładujemy funkcję celu
    p.set_objective( 
                    (R_1[0,0] * z[11_1] + R_1[0,1] * z[12_1] + R_1[0,2] * z[13_1]
                     + R_1[1,0] * z[21_1] + R_1[1,1] * z[22_1] + R_1[1,2] * z[23_1]
                     + R_1[2,0] * z[31_1] + R_1[2,1] * z[32_1] + R_1[2,2] * z[33_1])
                    )
    # Sprawdzamy wyniki
    p.show()
    print('Objective Value: {}'.format(p.solve()))

    print('Rozwiązanie: ')
    for i, v in sorted(p.get_values(z).items()):
        print(f'z_{i} = {v}')
    for i, v in sorted(p.get_values(q).items()):
        print(f'q_{i} = {v}')
    for i, v in sorted(p.get_values(a).items()):
        print(f'a_{i} = {v}')
    
test2()

macierz wypłat szeryfa
[[9 8 7]
 [6 5 4]
 [3 2 1]]
macierz wypłat kupca 1
[[1 2 3]
 [4 5 6]
 [7 8 9]]
Maximization:
  9.0 x_0 + 8.0 x_1 + 7.0 x_2 + 6.0 x_3 + 5.0 x_4 + 4.0 x_5 + 3.0 x_6 + 2.0 x_7 + x_8 

Constraints:
  1.0 <= x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 <= 1.0
  x_0 + x_1 + x_2 <= 1.0
  x_3 + x_4 + x_5 <= 1.0
  x_6 + x_7 + x_8 <= 1.0
  - x_0 - x_3 - x_6 + x_9 <= 0.0
  x_0 + x_3 + x_6 <= 1.0
  - x_1 - x_4 - x_7 + x_10 <= 0.0
  x_1 + x_4 + x_7 <= 1.0
  - x_2 - x_5 - x_8 + x_11 <= 0.0
  x_2 + x_5 + x_8 <= 1.0
  1.0 <= x_9 + x_10 + x_11 <= 1.0
  x_0 + x_1 + x_2 + 4.0 x_3 + 4.0 x_4 + 4.0 x_5 + 7.0 x_6 + 7.0 x_7 + 7.0 x_8 - x_12 <= 0.0
  - x_0 - x_1 - x_2 - 4.0 x_3 - 4.0 x_4 - 4.0 x_5 - 7.0 x_6 - 7.0 x_7 - 7.0 x_8 + 10000.0 x_9 + x_12 <= 10000.0
  2.0 x_0 + 2.0 x_1 + 2.0 x_2 + 5.0 x_3 + 5.0 x_4 + 5.0 x_5 + 8.0 x_6 + 8.0 x_7 + 8.0 x_8 - x_12 <= 0.0
  -2.0 x_0 - 2.0 x_1 - 2.0 x_2 - 5.0 x_3 - 5.0 x_4 - 5.0 x_5 - 8.0 x_6 - 8.0 x_7 - 8.0 x_8 + 10000.0 x_10 + x_12 <= 10000.

In [8]:
# Test sprawdzenia czy wszystko ok dla l=1
def game_l1(M=10000, a = [1], c=[1], legal=[2], illegal=[5]):
    
    # Macierz wypłat dla następnika
    C_1 = np.array([[6*(legal[0]+a[0]),-3*illegal[0] + 3*(legal[0]+a[0]),-6*illegal[0]],
                    [6*(legal[0]+a[0])-2+c[0],3*(illegal[0]+2*a[0])+3*(legal[0]+a[0])+c[0],6*(illegal[0]+2*a[0])+c[0]],
                    [6*(legal[0]+a[0])-2,3*(illegal[0]+2*a[0])+3*(legal[0]+a[0])-2,6*(illegal[0]+2*a[0])-2]])
    # Macierz wypłat dla lidera
    R_1 = np.array([[-6*(legal[0]),3*illegal[0],6*illegal[0]],
                    [2-c[0],5-c[0],8-c[0]],
                    [2,2-3,2-6]])

    print('macierz wypłat szeryfa')
    print(R_1)
    print('macierz wypłat kupca 1')
    print(C_1)

    # Setujemy problem
    p = MixedIntegerLinearProgram(maximization=True, solver="GLPK")
    
    # Dodajemy odpowiednie zmienne
    # Konwencja: z[12_3] to z dla i=1, j=2, l=3
    # q[2_1] to q dla j=2, l=1
    # a[1] to a dla l=1
    z = p.new_variable()
    p.set_min(z, 0)
    p.set_max(z, 1)
    q = p.new_variable(binary= True)
    a = p.new_variable()

    # Kolejno od góry do dołu zapisujemy ograniczenia
    p.add_constraint(z[11_1] + z[12_1] + z[13_1] + z[21_1] + z[22_1] + z[23_1] + z[31_1] + z[32_1] + z[33_1] == 1)


    p.add_constraint(z[11_1] + z[12_1] + z[13_1] <= 1)
    p.add_constraint(z[21_1] + z[22_1] + z[23_1] <= 1)
    p.add_constraint(z[31_1] + z[32_1] + z[33_1] <= 1)


    p.add_constraint(q[1_1] <= z[11_1] + z[21_1] + z[31_1]  <= 1)
    p.add_constraint(q[2_1] <= z[12_1] + z[22_1] + z[32_1]  <= 1)
    p.add_constraint(q[3_1] <= z[13_1] + z[23_1] + z[33_1]  <= 1)


    p.add_constraint(q[1_1] + q[2_1] + q[3_1]  == 1)


    p.add_constraint(0 <= a[1] - (C_1[0,0]*(z[11_1] + z[12_1] + z[13_1]) +
                              C_1[1,0]*(z[21_1] + z[22_1] + z[23_1]) +
                              C_1[2,0]*(z[31_1] + z[32_1] + z[33_1])) <= (1-q[1_1])*M)
    p.add_constraint(0 <= a[1] - (C_1[0,1]*(z[11_1] + z[12_1] + z[13_1]) +
                              C_1[1,1]*(z[21_1] + z[22_1] + z[23_1]) +
                              C_1[2,1]*(z[31_1] + z[32_1] + z[33_1])) <= (1-q[2_1])*M)
    p.add_constraint(0 <= a[1] - (C_1[0,2]*(z[11_1] + z[12_1] + z[13_1]) +
                              C_1[1,2]*(z[21_1] + z[22_1] + z[23_1]) +
                              C_1[2,2]*(z[31_1] + z[32_1] + z[33_1])) <= (1-q[3_1])*M)
    # ponieważ L = {1} to pomijamy ostatnie ogarniczenie
    
    # Ładujemy funkcję celu
    p.set_objective( 
                    (R_1[0,0] * z[11_1] + R_1[0,1] * z[12_1] + R_1[0,2] * z[13_1]
                     + R_1[1,0] * z[21_1] + R_1[1,1] * z[22_1] + R_1[1,2] * z[23_1]
                     + R_1[2,0] * z[31_1] + R_1[2,1] * z[32_1] + R_1[2,2] * z[33_1])
                    )
    # Sprawdzamy wyniki
    p.show()
    print('Objective Value: {}'.format(p.solve()))

    print('Rozwiązanie: ')
    for i, v in sorted(p.get_values(z).items()):
        print(f'z_{i} = {v}')
    for i, v in sorted(p.get_values(q).items()):
        print(f'q_{i} = {v}')
    for i, v in sorted(p.get_values(a).items()):
        print(f'a_{i} = {v}')
    
game_l1()

macierz wypłat szeryfa
[[-12  15  30]
 [  1   4   7]
 [  2  -1  -4]]
macierz wypłat kupca 1
[[ 18  -6 -30]
 [ 17  31  43]
 [ 16  28  40]]
Maximization:
  -12.0 x_0 + 15.0 x_1 + 30.0 x_2 + x_3 + 4.0 x_4 + 7.0 x_5 + 2.0 x_6 - x_7 -4.0 x_8 

Constraints:
  1.0 <= x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 <= 1.0
  x_0 + x_1 + x_2 <= 1.0
  x_3 + x_4 + x_5 <= 1.0
  x_6 + x_7 + x_8 <= 1.0
  - x_0 - x_3 - x_6 + x_9 <= 0.0
  x_0 + x_3 + x_6 <= 1.0
  - x_1 - x_4 - x_7 + x_10 <= 0.0
  x_1 + x_4 + x_7 <= 1.0
  - x_2 - x_5 - x_8 + x_11 <= 0.0
  x_2 + x_5 + x_8 <= 1.0
  1.0 <= x_9 + x_10 + x_11 <= 1.0
  18.0 x_0 + 18.0 x_1 + 18.0 x_2 + 17.0 x_3 + 17.0 x_4 + 17.0 x_5 + 16.0 x_6 + 16.0 x_7 + 16.0 x_8 - x_12 <= 0.0
  -18.0 x_0 - 18.0 x_1 - 18.0 x_2 - 17.0 x_3 - 17.0 x_4 - 17.0 x_5 - 16.0 x_6 - 16.0 x_7 - 16.0 x_8 + 10000.0 x_9 + x_12 <= 10000.0
  -6.0 x_0 - 6.0 x_1 - 6.0 x_2 + 31.0 x_3 + 31.0 x_4 + 31.0 x_5 + 28.0 x_6 + 28.0 x_7 + 28.0 x_8 - x_12 <= 0.0
  6.0 x_0 + 6.0 x_1 + 6.0 x_2 - 31.0 x_

In [12]:
# Właściwa symulacja gry
def game_l3(M=10000, a = [1,2,3], c=[6,5,4], legal=[2,4,2], illegal=[5,5,3], prob = [1/3,1/3,1/3]):
    
    # Macierze wypłat dla następników
    C_1 = np.array([[6*(legal[0]+a[0]),-3*illegal[0] + 3*(legal[0]+a[0]),-6*illegal[0]],
                    [6*(legal[0]+a[0])-2+c[0],3*(illegal[0]+2*a[0])+3*(legal[0]+a[0])+c[0],6*(illegal[0]+2*a[0])+c[0]],
                    [6*(legal[0]+a[0])-2,3*(illegal[0]+2*a[0])+3*(legal[0]+a[0])-2,6*(illegal[0]+2*a[0])-2]])
    C_2 = np.array([[6*(legal[1]+a[1]),-3*illegal[1] + 3*(legal[1]+a[1]),-6*illegal[1]],
                    [6*(legal[1]+a[1])-2+c[1],3*(illegal[1]+2*a[1])+3*(legal[1]+a[1])+c[1],6*(illegal[1]+2*a[1])+c[1]],
                    [6*(legal[1]+a[1])-2,3*(illegal[1]+2*a[1])+3*(legal[1]+a[1])-2,6*(illegal[1]+2*a[1])-2]])
    C_3 = np.array([[6*(legal[2]+a[2]),-3*illegal[2] + 3*(legal[2]+a[2]),-6*illegal[2]],
                    [6*(legal[2]+a[2])-2+c[2],3*(illegal[2]+2*a[2])+3*(legal[2]+a[2])+c[2],6*(illegal[2]+2*a[2])+c[2]],
                    [6*(legal[2]+a[2])-2,3*(illegal[2]+2*a[2])+3*(legal[2]+a[2])-2,6*(illegal[2]+2*a[2])-2]])
    
    # Macierze wypłat dla lidera
    R_1 = np.array([[-6*(legal[0]),3*illegal[0],6*illegal[0]],
                    [2-c[0],5-c[0],8-c[0]],[2,2-3,2-6]])
    R_2 = np.array([[-6*(legal[1]),3*illegal[1],6*illegal[1]],
                    [2-c[1],5-c[1],8-c[1]],[2,2-3,2-6]])
    R_3 = np.array([[-6*(legal[2]),3*illegal[2],6*illegal[2]],
                    [2-c[2],5-c[2],8-c[2]],[2,2-3,2-6]])

    print('macierz wypłat szeryfa vs kupiec 1')
    print(R_1)
    print('macierz wypłat kupca 1')
    print(C_1)
    print("")
    print('macierz wypłat szeryfa vs kupiec 2')
    print(R_2)
    print('macierz wypłat kupca 2')
    print(C_2)
    print("")
    print('macierz wypłat szeryfa vs kupiec 3')
    print(R_3)
    print('macierz wypłat kupca 3')
    print(C_3)

    # Setujemy problem
    p = MixedIntegerLinearProgram(maximization=True, solver="GLPK")
    
    # Dodajemy odpowiednie zmienne
    # Konwencja: z[12_3] to z dla i=1, j=2, l=3
    # q[2_1] to q dla j=2, l=1
    # a[1] to a dla l=1
    z = p.new_variable()
    p.set_min(z, 0)
    p.set_max(z, 1)
    q = p.new_variable(binary= True)
    a = p.new_variable()

    # Kolejno od góry do dołu zapisujemy ograniczenia
    p.add_constraint(z[11_1] + z[12_1] + z[13_1] + z[21_1] + z[22_1] + z[23_1] + z[31_1] + z[32_1] + z[33_1] == 1)
    p.add_constraint(z[11_2] + z[12_2] + z[13_2] + z[21_2] + z[22_2] + z[23_2] + z[31_2] + z[32_2] + z[33_2] == 1)
    p.add_constraint(z[11_3] + z[12_3] + z[13_3] + z[21_3] + z[22_3] + z[23_3] + z[31_3] + z[32_3] + z[33_3] == 1)

    p.add_constraint(z[11_1] + z[12_1] + z[13_1] <= 1)
    p.add_constraint(z[11_2] + z[12_2] + z[13_2] <= 1)
    p.add_constraint(z[11_3] + z[12_3] + z[13_3] <= 1)
    p.add_constraint(z[21_1] + z[22_1] + z[23_1] <= 1)
    p.add_constraint(z[21_2] + z[22_2] + z[23_2] <= 1)
    p.add_constraint(z[21_3] + z[22_3] + z[23_3] <= 1)
    p.add_constraint(z[31_1] + z[32_1] + z[33_1] <= 1)
    p.add_constraint(z[31_2] + z[32_2] + z[33_2] <= 1)
    p.add_constraint(z[31_3] + z[32_3] + z[33_3] <= 1)


    p.add_constraint(q[1_1] <= z[11_1] + z[21_1] + z[31_1]  <= 1)
    p.add_constraint(q[2_1] <= z[12_1] + z[22_1] + z[32_1]  <= 1)
    p.add_constraint(q[3_1] <= z[13_1] + z[23_1] + z[33_1]  <= 1)
    p.add_constraint(q[1_2] <= z[11_2] + z[21_2] + z[31_2]  <= 1)
    p.add_constraint(q[2_2] <= z[12_2] + z[22_2] + z[32_2]  <= 1)
    p.add_constraint(q[3_2] <= z[13_2] + z[23_2] + z[33_2]  <= 1)
    p.add_constraint(q[1_3] <= z[11_3] + z[21_3] + z[31_3]  <= 1)
    p.add_constraint(q[2_3] <= z[12_3] + z[22_3] + z[32_3]  <= 1)
    p.add_constraint(q[3_3] <= z[13_3] + z[23_3] + z[33_3]  <= 1)


    p.add_constraint(q[1_1] + q[2_1] + q[3_1]  == 1)
    p.add_constraint(q[1_2] + q[2_2] + q[3_2]  == 1)
    p.add_constraint(q[1_3] + q[2_3] + q[3_3]  == 1)


    p.add_constraint(0 <= a[1] - (C_1[0,0]*(z[11_1] + z[12_1] + z[13_1]) +
                              C_1[1,0]*(z[21_1] + z[22_1] + z[23_1]) +
                              C_1[2,0]*(z[31_1] + z[32_1] + z[33_1])) <= (1-q[1_1])*M)
    p.add_constraint(0 <= a[1] - (C_1[0,1]*(z[11_1] + z[12_1] + z[13_1]) +
                              C_1[1,1]*(z[21_1] + z[22_1] + z[23_1]) +
                              C_1[2,1]*(z[31_1] + z[32_1] + z[33_1])) <= (1-q[2_1])*M)
    p.add_constraint(0 <= a[1] - (C_1[0,2]*(z[11_1] + z[12_1] + z[13_1]) +
                              C_1[1,2]*(z[21_1] + z[22_1] + z[23_1]) +
                              C_1[2,2]*(z[31_1] + z[32_1] + z[33_1])) <= (1-q[3_1])*M)
    p.add_constraint(0 <= a[2] - (C_2[0,0]*(z[11_2] + z[12_2] + z[13_2]) +
                              C_2[1,0]*(z[21_2] + z[22_2] + z[23_2]) +
                              C_2[2,0]*(z[31_2] + z[32_2] + z[33_2])) <= (1-q[1_2])*M)
    p.add_constraint(0 <= a[2] - (C_2[0,1]*(z[11_2] + z[12_2] + z[13_2]) +
                              C_2[1,1]*(z[21_2] + z[22_2] + z[23_2]) +
                              C_2[2,1]*(z[31_2] + z[32_2] + z[33_2])) <= (1-q[2_2])*M)
    p.add_constraint(0 <= a[2] - (C_2[0,2]*(z[11_2] + z[12_2] + z[13_2]) +
                              C_2[1,2]*(z[21_2] + z[22_2] + z[23_2]) +
                              C_2[2,2]*(z[31_2] + z[32_2] + z[33_2])) <= (1-q[3_2])*M)
    p.add_constraint(0 <= a[3] - (C_3[0,0]*(z[11_3] + z[12_3] + z[13_3]) +
                              C_3[1,0]*(z[21_3] + z[22_3] + z[23_3]) +
                              C_3[2,0]*(z[31_3] + z[32_3] + z[33_3])) <= (1-q[1_3])*M)
    p.add_constraint(0 <= a[3] - (C_3[0,1]*(z[11_3] + z[12_3] + z[13_3]) +
                              C_3[1,1]*(z[21_3] + z[22_3] + z[23_3]) +
                              C_3[2,1]*(z[31_3] + z[32_3] + z[33_3])) <= (1-q[2_3])*M)
    p.add_constraint(0 <= a[3] - (C_3[0,2]*(z[11_3] + z[12_3] + z[13_3]) +
                              C_3[1,2]*(z[21_3] + z[22_3] + z[23_3]) +
                              C_3[2,2]*(z[31_3] + z[32_3] + z[33_3])) <= (1-q[3_3])*M)
    
    p.add_constraint(z[11_1] + z[12_1] + z[13_1] == z[11_2] + z[12_2] + z[13_2])
    p.add_constraint(z[11_1] + z[12_1] + z[13_1] == z[11_3] + z[12_3] + z[13_3])
    p.add_constraint(z[21_1] + z[22_1] + z[23_1] == z[21_2] + z[22_2] + z[23_2])
    p.add_constraint(z[21_1] + z[22_1] + z[23_1] == z[21_3] + z[22_3] + z[23_3])
    p.add_constraint(z[31_1] + z[32_1] + z[33_1] == z[31_2] + z[32_2] + z[33_2])
    p.add_constraint(z[31_1] + z[32_1] + z[33_1] == z[31_3] + z[32_3] + z[33_3])
    
    # Ładujemy funkcję celu
    p.set_objective(prob[0] * 
                    (R_1[0,0] * z[11_1] + R_1[0,1] * z[12_1] + R_1[0,2] * z[13_1]
                     + R_1[1,0] * z[21_1] + R_1[1,1] * z[22_1] + R_1[1,2] * z[23_1]
                     + R_1[2,0] * z[31_1] + R_1[2,1] * z[32_1] + R_1[2,2] * z[33_1])
                    +  prob[1] * (R_2[0,0] * z[11_2] + R_2[0,1] * z[12_2] + R_2[0,2] * z[13_2]
                     + R_2[1,0] * z[21_2] + R_2[1,1] * z[22_2] + R_2[1,2] * z[23_2]
                     + R_2[2,0] * z[31_2] + R_2[2,1] * z[32_2] + R_2[2,2] * z[33_2])
                    + prob[2] * (R_3[0,0] * z[11_3] + R_3[0,1] * z[12_3] + R_3[0,2] * z[13_3]
                     + R_3[1,0] * z[21_3] + R_3[1,1] * z[22_3] + R_3[1,2] * z[23_3]
                     + R_3[2,0] * z[31_3] + R_3[2,1] * z[32_3] + R_3[2,2] * z[33_3]))
    
    # Sprawdzamy wyniki
    p.show()
    print('Objective Value: {}'.format(p.solve()))

    print('Rozwiązanie: ')
    for i, v in sorted(p.get_values(z).items()):
        print(f'z_{i} = {v}')
    for i, v in sorted(p.get_values(q).items()):
        print(f'q_{i} = {v}')
    for i, v in sorted(p.get_values(a).items()):
        print(f'a_{i} = {v}')
    
game_l3(M=10000, a = [1,2,3], c=[1,5,4], legal=[2,4,2], illegal=[5,5,3], prob = [1,0,0])

macierz wypłat szeryfa vs kupiec 1
[[-12  15  30]
 [  1   4   7]
 [  2  -1  -4]]
macierz wypłat kupca 1
[[ 18  -6 -30]
 [ 17  31  43]
 [ 16  28  40]]

macierz wypłat szeryfa vs kupiec 2
[[-24  15  30]
 [ -3   0   3]
 [  2  -1  -4]]
macierz wypłat kupca 2
[[ 36   3 -30]
 [ 39  50  59]
 [ 34  43  52]]

macierz wypłat szeryfa vs kupiec 3
[[-12   9  18]
 [ -2   1   4]
 [  2  -1  -4]]
macierz wypłat kupca 3
[[ 30   6 -18]
 [ 32  46  58]
 [ 28  40  52]]
Maximization:
  -12.0 x_0 + 15.0 x_1 + 30.0 x_2 + x_3 + 4.0 x_4 + 7.0 x_5 + 2.0 x_6 - x_7 -4.0 x_8 

Constraints:
  1.0 <= x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 <= 1.0
  1.0 <= x_9 + x_10 + x_11 + x_12 + x_13 + x_14 + x_15 + x_16 + x_17 <= 1.0
  1.0 <= x_18 + x_19 + x_20 + x_21 + x_22 + x_23 + x_24 + x_25 + x_26 <= 1.0
  x_0 + x_1 + x_2 <= 1.0
  x_9 + x_10 + x_11 <= 1.0
  x_18 + x_19 + x_20 <= 1.0
  x_3 + x_4 + x_5 <= 1.0
  x_12 + x_13 + x_14 <= 1.0
  x_21 + x_22 + x_23 <= 1.0
  x_6 + x_7 + x_8 <= 1.0
  x_15 + x_16 + x_17 <= 1.0

<h4>Widzimy, że wynik game_l1 i game_l3 z odpowiadającymi jej ustawieniami zgadzają się.</h4>

Poniżej test dla domyślnych ustawień game_l3

In [13]:
game_l3()

macierz wypłat szeryfa vs kupiec 1
[[-12  15  30]
 [ -4  -1   2]
 [  2  -1  -4]]
macierz wypłat kupca 1
[[ 18  -6 -30]
 [ 22  36  48]
 [ 16  28  40]]

macierz wypłat szeryfa vs kupiec 2
[[-24  15  30]
 [ -3   0   3]
 [  2  -1  -4]]
macierz wypłat kupca 2
[[ 36   3 -30]
 [ 39  50  59]
 [ 34  43  52]]

macierz wypłat szeryfa vs kupiec 3
[[-12   9  18]
 [ -2   1   4]
 [  2  -1  -4]]
macierz wypłat kupca 3
[[ 30   6 -18]
 [ 32  46  58]
 [ 28  40  52]]
Maximization:
  -4.0 x_0 + 5.0 x_1 + 10.0 x_2 -1.3333333333333333 x_3 -0.3333333333333333 x_4 + 0.6666666666666666 x_5 + 0.6666666666666666 x_6 -0.3333333333333333 x_7 -1.3333333333333333 x_8 -8.0 x_9 + 5.0 x_10 + 10.0 x_11 - x_12 + x_14 + 0.6666666666666666 x_15 -0.3333333333333333 x_16 -1.3333333333333333 x_17 -4.0 x_18 + 3.0 x_19 + 6.0 x_20 -0.6666666666666666 x_21 + 0.3333333333333333 x_22 + 1.3333333333333333 x_23 + 0.6666666666666666 x_24 -0.3333333333333333 x_25 -1.3333333333333333 x_26 

Constraints:
  1.0 <= x_0 + x_1 + x_2 + x_3 + x